## Deep Information Retrieval and Neural Word Embeddings (Part 2)

### Outline:
7. Deep Learning
8. Word to Vector
9. Clustering
10. High-dimension Visualization

### Continue from part 1

#### Load the data we processed earlier

In [1]:
import pandas as pd
import numpy as np

## load a pickle file
train_df = pd.read_pickle("train_df.pkl")
test_df = pd.read_pickle("test_df.pkl")


import nltk
from sklearn.feature_extraction.text import CountVectorizer

BOW_500 = CountVectorizer(max_features=500, tokenizer=nltk.word_tokenize) 
BOW_500.fit(train_df['text'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=500, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x11920b9d8>, vocabulary=None)

---
## ＊ Before going down

### Some important tips on using machine learning library:
1. What are the input & ouput    


2. Read official document    


3. Understand the algorithm    

---
### 7. Deep Learning

We use [Keras](https://keras.io/) to be our deep learning framwork, and follow the [Model (functional API)](https://keras.io/models/model/) to build a Deep Neural Network (DNN) model.

If you want to use your computer to train, you have to install `keras` and `tensorflow`. 


Because Deep Learning is a 1-semester course, we can't talk about each detail about it in the lab session. Here, we only provide a simple template about how to build & run a DL model successfully. You can follow this template to design your model!


#### Fully Connected Network

![Image](https://stanford.edu/~shervine/images/neural-network.png)

(source: https://stanford.edu/~shervine/teaching/cs-229/cheatsheet-deep-learning)


#### Design model by ourself
1. Make sure the dimension of input & output.
2. Design the network architecture. (like the paper you read)

### 7-0 Import the library


In [2]:
# check keras & tensorflow work
import keras

Using TensorFlow backend.


### 7-1 Prepare data (X, y)

In [3]:
# standardize name (X, y) 
X_train = BOW_500.transform(train_df['text'])
y_train = train_df['emotion']

X_test = BOW_500.transform(test_df['text'])
y_test = test_df['emotion']

## check dimension is a good habbit 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape)
print('y_test.shape: ', y_test.shape)

X_train.shape:  (3613, 500)
y_train.shape:  (3613,)
X_test.shape:  (347, 500)
y_test.shape:  (347,)


### 7-2 Deal with categorical label (y)

Rather than put your label `train_df['emotion']` directly into a model, we have to process these categorical (or say nominal) label by ourselves. 

Here, we use the basic method [one-hot encoding](https://en.wikipedia.org/wiki/One-hot) to transform our categorical  labels to numerical ones.


In [4]:
## deal with label (string -> one-hot)

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train = label_encode(label_encoder, y_train)
y_test = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)


check label:  ['anger' 'fear' 'joy' 'sadness']

## Before convert
y_train[0:4]:
 1429       fear
3362    sadness
2035        joy
1255       fear
Name: emotion, dtype: object

y_train.shape:  (3613,)
y_test.shape:  (347,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]]

y_train.shape:  (3613, 4)
y_test.shape:  (347, 4)


### 7-3 Build model

In [5]:
# I/O check
input_shape = X_train.shape[1]
print('input_shape: ', input_shape)

output_shape = len(label_encoder.classes_)
print('output_shape: ', output_shape)

input_shape:  500
output_shape:  4


![Imgur](https://i.imgur.com/qt6ggCg.png)

In [6]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# show model construction
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                32064     
_________________________________________________________________
re_lu_1 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
re_lu_2 (ReLU)               (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260       
_________________________________________________________________
softmax_1 (Softmax)          (None, 4)                 0         
Total para

### 7-4 Training!

In [7]:
# training setting
epochs = 5
batch_size = 32

# training!
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
print('training finish')

Epoch 1/5
3613/3613 [==============================] - 1s 219us/step - loss: 1.3392 - acc: 0.3543
Epoch 2/5
3613/3613 [==============================] - 0s 105us/step - loss: 1.0215 - acc: 0.6333
Epoch 3/5
3613/3613 [==============================] - 0s 126us/step - loss: 0.6095 - acc: 0.7905
Epoch 4/5
3613/3613 [==============================] - 0s 124us/step - loss: 0.4250 - acc: 0.8445
Epoch 5/5
3613/3613 [==============================] - 1s 151us/step - loss: 0.3371 - acc: 0.8754
training finish


### 8-5 Predict on testing data

In [8]:
## predict
pred_result = model.predict(X_test, batch_size=128)
pred_result[:5]

array([[0.05938049, 0.7811237 , 0.01164773, 0.1478481 ],
       [0.31598797, 0.14650868, 0.3395147 , 0.19798861],
       [0.0900881 , 0.8219837 , 0.07173403, 0.01619418],
       [0.00133804, 0.00236331, 0.9803382 , 0.01596039],
       [0.02794855, 0.5485645 , 0.20416616, 0.2193208 ]], dtype=float32)

In [9]:
pred_result = label_decode(label_encoder, pred_result)
pred_result[:5]

array(['fear', 'joy', 'fear', 'joy', 'fear'], dtype=object)

In [10]:
from sklearn.metrics import accuracy_score

print('testing accuracy: {}'.format(round(accuracy_score(label_decode(label_encoder, y_test), pred_result), 2)))

testing accuracy: 0.73


---

### Note

If you don't have a GPU (level is higher than GTX 1060) or you are not good at setting lots of things about computer, we **highly recommend** you to use the [kaggle kernel](https://www.kaggle.com/kernels) to do deep learning model training. They have already installed all the librarys and provided free GPU for you to use.


### More Information for your reference

* Keras document: https://keras.io/
* Keras GitHub example: https://github.com/keras-team/keras/tree/master/examples
* CS229: Machine Learning: http://cs229.stanford.edu/syllabus.html
* Deep Learning cheatsheet: https://stanford.edu/~shervine/teaching/cs-229/cheatsheet-deep-learning
* Notes from Coursera Deep Learning courses: https://www.slideshare.net/TessFerrandez/notes-from-coursera-deep-learning-courses-by-andrew-ng
* If you want to try TensorFlow or PyTorch: https://github.com/omarsar/nlp_pytorch_tensorflow_notebooks
* If you find life is so hard: http://curricul.web.nthu.edu.tw/files/13-1073-11965.php

---
### 8. Word2Vector

We will introduce how to use `gensim` to train your word2vec model and how to load a pre-trained model.

https://radimrehurek.com/gensim/index.html

### 8-0 Import library

In [11]:
## check library
import gensim

In [12]:
## ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [13]:
# # if you want to see the training messages, you can use it
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### 8-1 Prepare training corpus

In [14]:
## the input type
train_df['text_tokenized'] = train_df['text'].apply(lambda x: nltk.word_tokenize(x))
train_df[['id', 'text', 'text_tokenized']].head()

,id,text,text_tokenized
1429,20572,@BlizzHeroes I feel horrible dealing with 'pla...,"[@, BlizzHeroes, I, feel, horrible, dealing, w..."
3362,40535,Hey @AppleSupport - how do I find my play hist...,"[Hey, @, AppleSupport, -, how, do, I, find, my..."
2035,30031,@ADenkyirah Happy birthday! Hope you have a wo...,"[@, ADenkyirah, Happy, birthday, !, Hope, you,..."
1255,20398,While we focus on issue of #IPCA @IHFOKids Ind...,"[While, we, focus, on, issue, of, #, IPCA, @, ..."
1104,20247,@dfkm1970 @tomddumba the struggle is real. Who...,"[@, dfkm1970, @, tomddumba, the, struggle, is,..."


In [15]:
training_corpus = train_df['text_tokenized'].values
training_corpus[:3]

array([list(['@', 'BlizzHeroes', 'I', 'feel', 'horrible', 'dealing', 'with', "'players", "'", 'in', 'HotS', '.', 'News', 'on', 'what', 'you', "'re", 'doing', 'about', 'it', '?', 'Getting', 'stuck', 'with', 'uncooperative', 'people', '!']),
       list(['Hey', '@', 'AppleSupport', '-', 'how', 'do', 'I', 'find', 'my', 'play', 'history', 'on', 'new', '#', 'ios10', '#', 'appleMusic', '#', 'music', '#', 'lost']),
       list(['@', 'ADenkyirah', 'Happy', 'birthday', '!', 'Hope', 'you', 'have', 'a', 'wonderful', 'day', 'filled', 'with', 'lots', 'of', 'joy', 'and', 'laughter', '&', 'lt', ';', '3', '(', 'despite', 'tumblr', 'being', 'a', 'jerk-', 'once', 'again', ')'])],
      dtype=object)

### 8-2 Training our model

More details: https://radimrehurek.com/gensim/models/word2vec.html

In [16]:
from gensim.models import Word2Vec

## setting
vector_dim = 100
window_size = 5
min_count = 1
training_iter = 20

## model
word2vec_model = Word2Vec(sentences=training_corpus, 
                          size=vector_dim, window=window_size, 
                          min_count=min_count, iter=training_iter)

![Imgur](https://i.imgur.com/Fca3MCs.png)

### 8-3 Word vector

In [17]:
# get the corresponding vector of a word
word_vec = word2vec_model.wv['happy']
word_vec

array([ 0.48818177, -0.12579677, -0.7732691 ,  0.14786604,  0.6977749 ,
        0.08214612,  0.13248982,  0.8265964 , -0.16473792, -1.1260434 ,
       -0.9533535 ,  0.15720157, -0.4951974 ,  0.18960436,  0.4468373 ,
        0.11603308,  0.48270628, -0.15770155,  0.54801625, -0.09724382,
        0.39148423, -0.5809302 ,  0.23284118,  0.00911151,  0.56651187,
        0.30745077,  0.86653274,  0.62118834, -0.8786046 ,  0.7718356 ,
       -0.46144414,  0.8072992 , -0.3739862 ,  0.08001085, -0.6234863 ,
       -0.22189187,  0.83449423,  0.73945   , -0.18855359, -0.32404888,
        0.34286866, -0.19258067,  0.28582466, -0.39198074, -0.4787329 ,
        0.37503076,  0.11268745,  0.71215963, -0.9166716 , -1.1023661 ,
       -0.06069747, -0.21996696,  0.46404794, -0.7494166 ,  0.525967  ,
        0.3317778 , -0.3753972 , -0.8337419 , -0.8789794 , -0.0236831 ,
        0.01487411,  0.52850634,  0.06171899,  0.62065154, -0.600755  ,
        0.8609399 ,  0.23093444,  0.43485293,  0.27247792,  0.68

In [18]:
word = 'happy'
topn = 10
word2vec_model.most_similar(word, topn=topn)

[('ukedchat', 0.9530709981918335),
 ('keeping', 0.9480637311935425),
 ('bday', 0.9463320970535278),
 ("'be", 0.944219708442688),
 ('plz', 0.940619707107544),
 ('help', 0.9355375170707703),
 ('tones', 0.935016393661499),
 ('A4', 0.9338036775588989),
 ('Read', 0.930770754814148),
 ('.\\nA', 0.9304399490356445)]

### 8-4 Using a pre-trained w2v model

#### (1) Download model by yourself

source: [GoogleNews-vectors-negative300](https://code.google.com/archive/p/word2vec/)

more details: https://radimrehurek.com/gensim/models/keyedvectors.html

In [ ]:
from gensim.models import KeyedVectors

## Note: this model is very huge, here costs lots of time
model_path = "./GoogleNews/GoogleNews-vectors-negative300.bin.gz"
w2v_google_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
print('load ok')

w2v_google_model.most_similar('happy', topn=10)

#### (2) Using gensim api

more pre-trained models: https://github.com/RaRe-Technologies/gensim-data

In [ ]:
import gensim.downloader as api

## If you see `SSL: CERTIFICATE_VERIFY_FAILED` error, use this:
import ssl
import urllib.request
ssl._create_default_https_context = ssl._create_unverified_context

glove_twitter_25_model = api.load("glove-twitter-25")
print('load ok')

glove_twitter_25_model.most_similar('happy', topn=10)

### 8-5 king + woman - man = ?

In [ ]:
w2v_google_model.most_similar(positive=['king', 'woman'], negative=['man'])

---
![Imgur](https://i.imgur.com/lXBIX1Y.png)

Now, We get the word vector, but our input data is a sequence of words (or say sentence). 

How can we utilize these "word" vectors to represent the sentence data and train our model?

---

### 9. Clustering: k-means

Here we introduce how to use `sklearn` to do the basic **unsupervised learning** approach, k-means.    

more details: http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html


#### Basic concept

![Image](https://i.imgur.com/PEdUf54.png)

(img source: https://towardsdatascience.com/k-means-clustering-identifying-f-r-i-e-n-d-s-in-the-world-of-strangers-695537505d)

![Imgur](https://i.imgur.com/lM7aigN.png)

In [ ]:
# clustering target
target_list = ['happy', 'fear', 'angry', 'car', 'teacher', 'computer']
print('target words: ', target_list)

# convert to word vector
X = [word2vec_model.wv[word] for word in target_list]

In [ ]:
from sklearn.cluster import KMeans

# we have to decide how many cluster (k) we want
k = 2

# k-means model
kmeans_model = KMeans(n_clusters=k)
kmeans_model.fit(X)

# cluster result
cluster_result = kmeans_model.labels_

# show
for i in range(len(target_list)):
    print('word: {} \t cluster: {}'.format(target_list[i], cluster_result[i]))

![Imgur](https://i.imgur.com/JeZ27M2.png)

In [ ]:
word = 'student'
word_vec = word2vec_model.wv[word]
kmeans_model.predict([word_vec])

In [ ]:
word = 'sad'
word_vec = word2vec_model.wv[word]
kmeans_model.predict([word_vec])

---
![Imgur](https://i.imgur.com/lXBIX1Y.png)

As this example, although our task is a classification problem, clustering algorithm would help us to find some hidden relations automatically.  
In other words, utilizing this technique, you may extract useful features from our data. 

Unfolding your creativity to do more feature engineering! [Hint: 😊 😡 🤫 😇]

---


### 10. High-dimension Visualization: t-SNE

No matter you use the Bag-of-words, tf-idf, or word2vec, it's very hard to see the embedding result, because the dimension is larger than 3.  

In Lab 1, we already talked about PCA. We can use PCA to reduce the dimension of our data, then visialize them.  
However, if you dig deeper into the result, you should find it is insufficient...

source: https://www.fabian-keller.de/research/high-dimensional-data-visualization 
![Imgur](https://i.imgur.com/6IrSODM.png)


Here we would like to introduce another visualization method called t-SNE.  
http://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html 

### 10-0 Prepare visualize target

prepare data lists like:
![Imgur](https://i.imgur.com/vKkJ8sw.png)

In [ ]:
word_list = ['happy', 'angry', 'data', 'mining']

topn = 5
happy_words = ['happy'] + [word_ for word_, sim_ in w2v_google_model.most_similar('happy', topn=topn)]
angry_words = ['angry'] + [word_ for word_, sim_ in w2v_google_model.most_similar('angry', topn=topn)]        
data_words = ['data'] + [word_ for word_, sim_ in w2v_google_model.most_similar('data', topn=topn)]        
mining_words = ['mining'] + [word_ for word_, sim_ in w2v_google_model.most_similar('mining', topn=topn)]        

print('happy_words: ', happy_words)
print('angry_words: ', angry_words)
print('data_words: ', data_words)
print('mining_words: ', mining_words)

target_words = happy_words + angry_words + data_words + mining_words
print('\ntarget words: ')
print(target_words)

print('\ncolor list:')
cn = topn + 1
color = ['b'] * cn + ['g'] * cn + ['r'] * cn + ['y'] * cn
print(color)

### 10-1 Plot by t-SNE (2-dimension)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

## w2v model
model = w2v_google_model

## prepare training word vectors
size = 200
target_size = len(target_words)
all_word = list(model.vocab.keys())
word_train = target_words + all_word[:size]
X_train = model[word_train]

## t-SNE model
tsne = TSNE(n_components=2, metric='cosine', random_state=28)

## training
X_tsne = tsne.fit_transform(X_train)

## plot the result
plt.figure(figsize=(7.5, 7.5), dpi=115)
plt.scatter(X_tsne[:target_size, 0], X_tsne[:target_size, 1], c=color)
for label, x, y in zip(target_words, X_tsne[:target_size, 0], X_tsne[:target_size, 1]):
    plt.annotate(label, xy=(x,y), xytext=(0,0),  textcoords='offset points')
plt.show()

---
![Imgur](https://i.imgur.com/lXBIX1Y.png)

![Imgur](https://i.imgur.com/rDT04Ml.png)


---

---

![Imgur](https://i.imgur.com/MH3DkUq.png)

Assignment 2 competition link: [here](https://www.kaggle.com/c/datamininglab2)     
(you can not join by this link, we will announce the private link on iLMS)

### 1. Scoring

* Kaggle competition part **(40%)**:
![Imgur](https://i.imgur.com/EsMyKDS.png)   
    
   

* Competition report part **(60%)**:  
The score of your report will base on: readability, completeness, and creativity.

### 2. Important Dates

* 2018/11/05 (Mon) 12:30 p.m. - Start of competition


* 2018/11/25 (Sun) 23:00 p.m. - End of competition, final score announcement


* 2018/11/28 (Wed) 23:59 p.m. - Report submission (iLMS)

---

## Part 1: Kaggle competition

* Competition link will be announced on iLMS, only this course's student can involve, please don't share to others.   
  
  
* This is an indicidual competition, everyone is a team.    
  
  
* Before the end of the competition, you can only see the `Public Leaderboard` (Your score for public data).   
But your real score is decided by the private data, which is reveal on `Private Leaderboard` till the end of the competition.  
Try to avoid overfitting on public testing data.  
  
  
*  You are **not allowed** to use any pre-trained models or other datasets. But you can train any models from scratch by yourself.  
  
  
* You are **not allowed** to share your code to other classmate, but you could exchange your idea.  
  
  
* Other information are in our competition website.


---
## Part 2: Competition report

* Write and organize your report with jupyter notebook and upload it to iLMS.   
(The file name of your report must be `DM_hw2_{Your student ID}.ipynb`.  e.g DM_hw2_106065501.ipynb )
   
   
* In your report, you **need** to provide:
    * Your **student ID, name, team name, snapshot** of your `Private Leaderboard` score.  
    (Put these at the beginning of your report, or you will get **"0"**)   
         
    * code   
    (All the code which start from "load data" to "generate the file that submit to Kaggle") 
    
    * comments (short notes or explaination for your code)
    
   
* In your report, you **need** to include:
    * Descript whole flow/pipeline of your work  
      
    * How did you pre-process/clean your data
     
    * What feature engineering methods have you tried  
    
    * What model have you used
    
    * Did you encounter overfitting or underfitting problem? How did you against it
    
    * Conclusion
    

* In your report, it would be great to tell us:
    * Any special model and process method you tried during your experiments  
      
    * Share any interesting findings, feeling or suggestions   
      
    * compare different models and results   
    (store each experiment you have tried is a gooooog habit)

        
* Please, please, please....submit a "readable" jupyter notebook. (or we will deduct some score this time.)  
